# Нейросеть для определения эмоций по ключевым точкам

In [12]:
from matplotlib import pylab
from six.moves import cPickle as pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plot

Загрузим подготовленные ранее данные

In [14]:
pickle_file = 'Emotions dataset.pickle'

with open(pickle_file, 'rb') as f:
    data = pickle.load(f)
    train_dataset = data['train_data']
    train_labels = data['train_labels']
    test_dataset = data['test_data']
    test_labels = data['test_labels']
    del data
    f.close()

Загрузим некоторые константы

In [19]:
number_of_steps = 5001
beta = 0.005
sigma = 5e-2
init_learning_rate = 0.002
dropout_value_1 = 0.75
dropout_value_2 = 0.85
dropout_value_3 = 0.9
hidden_layer_1 = 1200
hidden_layer_2 = 720
hidden_layer_3 = 400
batch_size = 128
decay_steps = 1000
points_count = 68-17
labels_count = 7
decay_rate = 0.9
INF = 1e9 + 7
mINF = -1e9 - 7

In [15]:
print("Размер тренировочных данных:", train_dataset.shape)
print("Размер тестовых данных:", test_dataset.shape)

Размер тренировочных данных: (1400, 102)
Размер тестовых данных: (145, 102)


### Опишем граф операций, по которым работает наша нейронная сеть

In [16]:
graph = tf.Graph()

with graph.as_default():
   
    # Загружаем тестовые и тренировочные данные
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, points_count * 2))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, labels_count))
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)

    # Инициализируем матрицы весов
    weights1 = tf.Variable(
                tf.truncated_normal([points_count * 2, hidden_layer_1], stddev = sigma))
    weights2 = tf.Variable(
                tf.truncated_normal([hidden_layer_1, hidden_layer_2], stddev = sigma))
    weights3 = tf.Variable(
                tf.truncated_normal([hidden_layer_2, hidden_layer_3], stddev = sigma))
    weights4 = tf.Variable(
                tf.truncated_normal([hidden_layer_3, labels_count], stddev = sigma))
    
    # Инициализируем веса для нейронов смещения
    biases1 = tf.Variable(
                tf.constant(0.1, shape = (hidden_layer_1,)))
    biases2 = tf.Variable(
                tf.constant(0.1, shape = (hidden_layer_2,)))
    biases3 = tf.Variable(
                tf.constant(0.1, shape = (hidden_layer_3,)))
    biases4 = tf.Variable(
                tf.constant(0.1, shape = (labels_count,)))
    
    # Описываем модель
    def model(input, p1, p2, p3):
        hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(input, weights1) + biases1), p1)
        hidden2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden1, weights2) + biases2), p2)
        hidden3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden2, weights3) + biases3), p3)
        logits = tf.nn.bias_add(tf.matmul(hidden3, weights4), biases4)
        return logits
    
    # Вычисляем ошибку по формуле перекрёстной энтропии
    logits = model(tf_train_dataset, dropout_value_1, dropout_value_2, dropout_value_3)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels)) + (
        tf.nn.l2_loss(weights1)
        + tf.nn.l2_loss(weights2)
        + tf.nn.l2_loss(weights3)) * beta
    
    # Уменьшаем ошибку, уменьшая скорость обучения на каждом шаге
    global_step = tf.Variable(0, dtype = tf.int64)
    learning_rate = tf.train.exponential_decay(init_learning_rate, global_step, decay_steps, decay_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate, 0.9, 0.999, 1e-08).minimize(loss, global_step = global_step)
    model_saver = tf.train.Saver()
    
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(model(test_dataset, 1.0, 1.0, 1.0))

In [ ]:
line = '-----------------'

def accuracy(predictions, labels):
    array1 = np.argmax(predictions, 1)
    array2 = np.argmax(labels, 1)
    sumi = np.sum(array1 == array2)
    return (100.0 * sumi / predictions.shape[0])

with tf.Session(graph = graph) as session:

    tf.global_variables_initializer().run()
    print('Обучение началось')
    
    for step in range(number_of_steps):
        
        # Данные в нейросеть будем загружать небольшими пакетами по 128 примеров
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        o, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 1000 == 0):
            print(line)
            print('Ошибка на шаге %d: %f' % (step, l))
            print('Точность на тренировочных данных: %.1f%%' % accuracy(predictions, batch_labels))
            print('Точность на тестовых данных: %.1f%%' % accuracy(test_prediction.eval(), test_labels))